# Detail Rocket Deep Learning Recommendation System

## Imports and installs

In [51]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
#%pip install pandas
# %pip install seaborn
#%pip install numpy

from tensorflow.python.ops.gen_array_ops import ExpandDims
from keras.layers import RepeatVector, TimeDistributed, Layer, Reshape, BatchNormalization, Attention, Input, Embedding, LSTM, Dense, Dot, Activation, Concatenate, Flatten

from keras.models import Model

In [52]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from keras.utils import pad_sequences
#test

## Load Data

Preprocessing was done in EDA notebook and stored in pickle files

In [53]:
basepath = '../../datasets/preprocessed_datasets/retailrocket/'
category_tree = pd.read_pickle(basepath +'category_tree.pkl')
category_tree

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
...,...,...
1664,49,1125.0
1665,1112,630.0
1666,1336,745.0
1667,689,207.0


In [54]:
events = pd.read_pickle(basepath +'events.pkl')
events

,timestamp,visitorid,event,itemid,transactionid,datetime
0,1433221332117,257597,view,355908,NaN,2015-06-02 07:02:12
1,1433224214164,992329,view,248676,NaN,2015-06-02 07:50:14
2,1433221999827,111016,view,318965,NaN,2015-06-02 07:13:19
3,1433221955914,483717,view,253185,NaN,2015-06-02 07:12:35
4,1433221337106,951259,view,367447,NaN,2015-06-02 07:02:17
...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,2015-08-01 05:13:05
2756097,1438399813142,762376,view,115946,NaN,2015-08-01 05:30:13
2756098,1438397820527,1251746,view,78144,NaN,2015-08-01 04:57:00
2756099,1438398530703,1184451,view,283392,NaN,2015-08-01 05:08:50


In [55]:
visitor_actions = pd.read_pickle(basepath +'visitor_actions.pkl')
visitor_actions

,visitorid,products_viewed,num_views,num_add2cart,num_transaction
0,693516,1,1,2,0
1,829044,1,1,0,0
2,652699,1,1,0,0
3,1125936,1,1,0,0
4,1149227,1,1,0,0
...,...,...,...,...,...
1407575,998517,1,1,0,0
1407576,472345,1,1,0,0
1407577,1207677,1,1,0,0
1407578,255126,1,1,0,0


In [56]:
product_item_df = pd.read_pickle(basepath +'product_item.pkl')
product_item_df

,itemid,visitorid,view,addtocart,transaction
0,3,370720,1.0,0,0
1,3,639016,1.0,0,0
2,4,905555,1.0,0,0
3,4,1010132,1.0,0,0
4,4,1042455,1.0,0,0
...,...,...,...,...,...
2145174,466864,1388296,4.0,0,0
2145175,466867,241803,1.0,0,0
2145176,466867,450511,1.0,0,0
2145177,466867,1001859,1.0,0,0


In [57]:
product_item_df[product_item_df.view >= 2]

,itemid,visitorid,view,addtocart,transaction
10,6,330981,2.0,0,0
18,6,771857,2.0,0,0
19,6,802121,2.0,0,0
36,15,724401,2.0,0,0
40,15,989647,2.0,0,0
...,...,...,...,...,...
2145162,466864,1076431,3.0,0,0
2145165,466864,1183853,6.0,0,0
2145167,466864,1224868,2.0,0,0
2145169,466864,1265478,3.0,0,0


In [58]:
item_properties = pd.read_pickle(basepath +'item_data_extracted.pkl')
item_properties

,itemid,categoryid,available,properties,property_values
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189,...","[1276750, n3.168 1144008, n15360.000 628176 n1..."
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 2...","[769062, 1154859, 679677, 519769, 769062, 1726..."
2,2,1305,0,"[1063, 112, 159, 202, 282, 283, 318, 332, 364,...","[n396.000 145688, 679677, 519769, 648485 n6000..."
3,3,1171,0,"[1025, 1080, 112, 159, 202, 227, 250, 283, 30,...","[769062, 769062, 679677, 519769, 261419, 13057..."
4,4,1038,0,"[112, 115, 159, 202, 227, 28, 283, 33, 364, 59...","[679677, n24.000, 519769, 371058 71429, 588652..."
...,...,...,...,...,...
417048,466862,1376,0,"[112, 139, 159, 186, 202, 227, 28, 283, 29, 34...","[679677, 769062, 519769, 575816 n432.000, 7173..."
417049,466863,173,0,"[1014, 112, 159, 202, 227, 283, 364, 400, 47, ...","[1075463, 679677, 519769, 1194687 550565 24255..."
417050,466864,373,1,"[1036, 112, 152, 159, 202, 227, 230, 283, 348,...","[1154859, 679677, 1071492, 519769, 1262739 205..."
417051,466865,421,0,"[1031, 112, 159, 202, 227, 277, 28, 283, 332, ...","[1088309, 679677, 519769, 150169 780351 820477..."


In [59]:
rebuild_grouped_df = False
if  rebuild_grouped_df:
    merged_df = pd.merge(item_properties, product_item_df, on='itemid')

    grouped_df = merged_df.groupby('visitorid').agg({
        'itemid': list,
        'categoryid': list,
        'properties': list,
        'property_values': list,
        'available': list,
        'view': list,
        'addtocart': list,
        'transaction': list
    }).reset_index()
    grouped_df.columns = ['visitorid', 'itemids', 'categories', 'properties', 'property_values', 'available', 'view', 'addtocart', 'transaction']

    grouped_df.to_pickle(basepath +'visitor_items.pkl')
    visitor_items = grouped_df
else:
    visitor_items = pd.read_pickle(basepath +'visitor_items.pkl')

visitor_items
# Select the desired columns0

,visitorid,itemids,categories,properties,property_values,available,view,addtocart,transaction
0,0,"[67045, 285930, 357564]","[333, 1188, 256]","[[1036, 112, 159, 202, 227, 28, 283, 328, 364,...","[[1154859, 679677, 519769, 439698, 1037547 150...","[1, 0, 1]","[1.0, 1.0, 1.0]","[0, 0, 0]","[0, 0, 0]"
1,1,[72028],[1192],"[[1036, 112, 159, 202, 227, 230, 283, 307, 364...","[[1318567, 679677, 519769, 72800, 887448 17434...",[0],[1.0],[0],[0]
2,2,"[216305, 259884, 325215, 342816]","[299, 299, 299, 444]","[[1030, 1036, 112, 159, 202, 227, 27, 283, 364...","[[140221, 961511, 679677, 519769, 961487 75577...","[1, 1, 0, 1]","[2.0, 1.0, 3.0, 2.0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
3,3,[385090],[1171],"[[1025, 1080, 112, 159, 202, 227, 250, 283, 30...","[[n464.400 357845 903043, [215470 n9048.000, 2...",[0],[1.0],[0],[0]
4,5,[61396],[646],"[[1092, 112, 159, 202, 283, 348, 364, 461, 506...","[[291010, 679677, 519769, 494986 789585, 72671...",[0],[1.0],[0],[0]
...,...,...,...,...,...,...,...,...,...
1236027,1407575,[121220],[1221],"[[112, 159, 202, 206, 227, 28, 283, 364, 476, ...","[[679677, 519769, 961131 584779 30834, n600.00...",[0],[1.0],[0],[0]
1236028,1407576,[356208],[175],"[[1075, 112, 159, 202, 25, 283, 364, 6, 678, 6...","[[[n120.000 1029109 n1200.000 1029109, n120.00...",[1],[1.0],[0],[0]
1236029,1407577,[427784],[1237],"[[112, 159, 19, 202, 227, 28, 283, 325, 364, 5...","[[679677, 519769, 1297729 n36.000 350726 30603...",[0],[1.0],[0],[0]
1236030,1407578,[188736],[1584],"[[112, 159, 202, 227, 28, 283, 364, 6, 678, 69...","[[679677, 519769, 727751 n6540.000 253573, 104...",[0],[1.0],[0],[0]


### build and train model

In [124]:
df = visitor_items.sample(n=10000)
# df = visitor_items.copy()

def convertToArray(column):
    return column.apply(lambda x: np.array([x]))

def bringToSameSizeOneDim(column, max_length = None):
    if max_length is None:
        max_length = max(column.apply(len))
    return column.apply(lambda x: x + [0] * (max_length - len(x)))


def bringToSameSizeTwoDimAndFlatten(column):
    max_length = max(column.apply(lambda x: max(len(sublist) for sublist in x)))
    new_column = column.apply(lambda x: [sublist + [0] * (max_length - len(sublist)) for sublist in x])
    new_column = new_column.apply(lambda x: x + [[0] * max_length] * (max_length - len(x)))
    return new_column.apply(lambda x: [item for sublist in x for item in sublist])
# itemid,categoryid,available,properties,property_values

def convertToSequence(column, max_length = None):
    list = column.tolist()
    if max_length is None:
        max_length = max(len(x) for x in list)
    return pad_sequences(list, maxlen=max_length, padding='post', value=0), max_length



itemids,max_itemid = convertToSequence(df.itemids)
categories,max_category = convertToSequence(df.categories)
available,max_available = convertToSequence(df.available)
view,max_view = convertToSequence(df.view)
addtocart,max_addtocart = convertToSequence(df.addtocart)

max_seq_length = max( max_itemid, max_category, max_available, max_view, max_addtocart)

visitorids, max_visitorid = convertToSequence(bringToSameSizeOneDim(convertToArray(df.visitorid), max_seq_length+1))
# to predict
transaction,max_transaction = convertToSequence(bringToSameSizeOneDim(df.transaction))
num_transactions = transaction.shape[1]

max_visitorid_id = max(np.unique(df.visitorid)) + 1
# max_visitorid = max_visitorid_id

max_itemid_id = np.max(np.unique(df.itemids.apply(np.unique).apply(max))) + 1
max_category_id = max(np.unique(df.categories.apply(np.unique).apply(max).astype(int))) + 1
max_available_id = int(max(np.unique(df.available.apply(np.unique).apply(max)))) + 1
max_view_id = int(max(np.unique(df.view.apply(np.unique).apply(max)))) + 1
max_addtocart_id = int(max(np.unique(df.addtocart.apply(np.unique).apply(max)))) + 1
max_transaction_id = int(max(np.unique(df.addtocart.apply(np.unique).apply(max)))) + 1




In [125]:
# Define input layers
input_visitorids = Input(shape=(max_visitorid,), name='visitorids')
input_itemids = Input(shape=(max_itemid,), name='itemids')
input_categories = Input(shape=(max_category,), name='categories')
# input_properties = Input(shape=(max_seq_length,), name='properties')
# input_property_values = Input(shape=(max_seq_length,), name='property_values')
input_available = Input(shape=(max_available,), name='available')
input_view = Input(shape=(max_view,), name='view')
input_addtocart = Input(shape=(max_addtocart,), name='addtocart')
# input_transaction = Input(shape=(1,), name='transaction')

encoding_padding_mask = tf.math.logical_not(tf.math.equal(input_itemids, 0))

# Embedding layers for categorical features
embedding_dim = 32
embedding_visitorids = Embedding(input_dim=max_visitorid_id, output_dim=embedding_dim)(input_visitorids)
embedding_itemids = Embedding(input_dim=max_itemid_id, output_dim=embedding_dim)(input_itemids)
embedding_categories = Embedding(input_dim=max_category_id, output_dim=embedding_dim)(input_categories)
embedding_available = Embedding(input_dim=max_available_id, output_dim=embedding_dim)(input_available)
embedding_view = Embedding(input_dim=max_view_id, output_dim=embedding_dim)(input_view)
embedding_addtocart = Embedding(input_dim=max_addtocart_id, output_dim=embedding_dim)(input_addtocart)
#embedding_transaction = Embedding(input_dim=max_category, output_dim=embedding_dim)(input_transaction)
# embedding_properties = Embedding(input_dim=max_property + 1, output_dim=embedding_dim)(input_properties)
# embedding_property_values = Embedding(input_dim=max_property_value + 1, output_dim=embedding_dim)(input_property_values)

# Concatenate all embeddings
concat_embedding_input = Concatenate(
    axis=-1, name='concat_embedding_input')([embedding_visitorids, embedding_itemids, embedding_categories, embedding_available, embedding_view, embedding_addtocart])

batchnorm_inputs = BatchNormalization(
    name='batchnorm_inputs')(concat_embedding_input)
# LSTM layer
lstm = LSTM(64, return_sequences=True)(concat_embedding_input)
lstm = BatchNormalization(name='batchnorm_lstm')(lstm)

# Apply attention mechanism
att = Attention(use_scale=False, name='attention')(inputs=[lstm, lstm],
                                                   mask=[encoding_padding_mask, encoding_padding_mask])
flatten = Flatten()(att)
output = Dense(num_transactions)(flatten)

model = Model(inputs=[input_visitorids, input_itemids, input_categories, input_available, input_view, input_addtocart], outputs=output)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', 'accuracy'])
model.summary()


Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 visitorids (InputLayer)        [(None, 523)]        0           []                               
                                                                                                  
 itemids (InputLayer)           [(None, 523)]        0           []                               
                                                                                                  
 categories (InputLayer)        [(None, 523)]        0           []                               
                                                                                                  
 available (InputLayer)         [(None, 523)]        0           []                               
                                                                                           

In [127]:
epochs = 2
batch_size = 32

history = model.fit([visitorids, itemids, categories, available, view, addtocart], transaction, epochs=epochs, batch_size=batch_size)  # Adjust epochs and batch size as needed

Epoch 1/2
313/313 [==============================] - 134s 428ms/step - loss: 0.0574 - mse: 0.0574 - mae: 0.0407 - accuracy: 0.0529
Epoch 2/2
313/313 [==============================] - 126s 403ms/step - loss: 25.1039 - mse: 25.1039 - mae: 0.2718 - accuracy: 0.0118


In [128]:
history.history

{'loss': [0.05743969604372978, 25.10390853881836],
 'mse': [0.05743969604372978, 25.10390853881836],
 'mae': [0.040722839534282684, 0.27175173163414],
 'accuracy': [0.05290000140666962, 0.011800000444054604]}

In [129]:
model.save('../../models/retailrocket/attention_detailrocket_v1.h5')

In [135]:
def preprocess_df(df, max_seq_length):
    itemids,max_itemid = convertToSequence(df.itemids, max_seq_length)
    categories,max_category = convertToSequence(df.categories, max_seq_length)
    available,max_available = convertToSequence(df.available, max_seq_length)
    view,max_view = convertToSequence(df.view, max_seq_length)
    addtocart,max_addtocart = convertToSequence(df.addtocart, max_seq_length)
    max_seq_length = max( max_itemid, max_category, max_available, max_view, max_addtocart)

    visitorids, max_visitorid = convertToSequence(bringToSameSizeOneDim(convertToArray(df.visitorid), max_seq_length+1))

    return visitorids, itemids, categories, available, view, addtocart

In [136]:
data_predict = preprocess_df(df.sample(1), max_seq_length)
data_predict

(array([[1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1184882,
         1184882, 1184882, 1184882, 1184882, 1184882, 1184882, 1

In [138]:
result = model.predict(data_predict)
result

1/1 [==============================] - 0s 41ms/step


array([[-1.022852  , -1.0886898 ,  1.1256257 ,  1.1768901 , -1.0393275 ,
         1.1031483 ,  1.0851078 ,  2.1013749 , -1.0069191 , -1.0533003 ,
         1.0742866 , -1.0712855 ,  1.0894234 , -0.9403493 ,  1.0128205 ,
        -1.0082606 ,  1.0448794 , -1.1097534 ,  1.0491636 ,  0.96531236,
         1.0564204 ,  1.1756315 ,  1.0819259 ,  2.0747838 , -1.058313  ,
         1.1073779 ,  1.0236592 , -1.0368581 ,  1.0930792 ,  1.0896103 ,
         1.0689301 , -1.1804254 , -1.0304452 ,  1.0948484 ,  1.0662332 ,
         1.016315  ,  1.3339212 ,  1.1190012 ,  1.0582678 , -1.0384842 ,
         1.0756857 ,  1.0371089 , -1.0844923 ,  0.9906556 ,  1.0122288 ,
         2.0667202 ,  1.0526197 ,  1.0396066 ,  1.0878049 ,  1.0447487 ,
        -1.0840561 ,  1.1000922 ,  1.0424135 ,  1.0664696 , -1.0374798 ,
         1.0883851 ,  1.0650017 ,  1.0661006 ,  2.0692542 , -1.0644532 ,
        -1.0248207 ,  1.3651495 ,  1.1830636 , -1.0167512 , -1.0412135 ,
        -1.0958662 , -1.0320467 , -1.1462679 ,  1.0

# Prepare data for model